In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import xgboost as xgb
import random

In [2]:
data = pd.read_csv("dollar_data/dollar_data_SIGNALS.csv")

In [3]:
index = 0
max_days = 2
#data["High"].loc[[(data.index) == i for i in range(index, index+max_days-1)]].max()
data["High"][index:index+max_days-1].max()

966.09

In [4]:
def standard_labeling(data):
    predict_data = data.copy().drop(["Open","Close","High","Low"],axis=1)
    max_days = 5
    target_range = 2
    for i in range(1,max_days):#2jours
        #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG"]].shift(i)
        #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i), "ADX-{}".format(i), "+DM-{}".format(i), "-DM-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG","ADX14","+DM","-DM"]].shift(i)
        predict_data[["Variation-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","RSI","MACD","MACD_H"]].shift(i)
    #predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
    predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
    predict_data["Target"] = np.where(predict_data["Target"] == True, 1, 0)
    predict_data.dropna(inplace=True)
    predict_data.reset_index(inplace=True,drop=True)
    predict_data = predict_data[0:len(predict_data)-target_range]

    return predict_data

In [10]:
def meta_labeling(data):
    predict_data = data.copy().drop(["Open","Close","High","Low"],axis=1)
    max_days = 5
    target_range = 2
    size = data.shape[0]
    first = None
    for i in range(1,max_days):#2jours
        #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG"]].shift(i)
        #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i), "ADX-{}".format(i), "+DM-{}".format(i), "-DM-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG","ADX14","+DM","-DM"]].shift(i)
        predict_data[["Variation-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","RSI","MACD","MACD_H"]].shift(i)
    #predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
    
#    for row in data.itertuples(index=True):
#        index = row[0]
    for index,row in data.iterrows():
        max_range = data[["Unix","High"]].loc[data["High"] == data["High"][index:index+max_days-1].max()].reset_index(drop=True)
        min_range = data[["Unix","Low"]].loc[data["Low"] == data["Low"][index:index+max_days-1].min()].reset_index(drop=True)
        max_unix = -np.inf
        min_unix = np.inf

        if (max_range["High"].loc[0]/data["Close"].loc[index]) > 1.02:
            max_unix = max_range["Unix"].loc[0]
        if (min_range["Low"].loc[0]/data["Close"].loc[index]) < 0.08:
            min_unix = min_range["Unix"].loc[0]

        if max_unix == -np.inf and min_unix == np.inf:
            predict_data.at[index, "Target"] = 0

        else:
            if min_unix < max_unix:
                first = -1
            else:
                first = 1
        predict_data.at[index, "Target"] = first

        if index % 1000 == 0:
            print("Itération {}/{}".format(index,size),end='\r')
    
    predict_data["Target1"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
    predict_data["Target1"] = np.where(predict_data["Target1"] == True, 1, 0)
    predict_data.dropna(inplace=True)
    predict_data.reset_index(inplace=True,drop=True)
    predict_data = predict_data[0:len(predict_data)-target_range]

    return predict_data

In [11]:
target_range = 2
max_s = data[["Unix", "Date", "High"]].rolling(target_range, on="High").max()
min_s = data[["Unix", "Date", "Low"]].rolling(target_range, on="Low").min()
min_s

,Unix,Low
0,NaN,962.54
1,1.483233e+09,962.54
2,1.483233e+09,960.53
3,1.483238e+09,960.53
4,1.483243e+09,962.80
...,...,...
597310,1.646101e+09,42998.08
597311,1.646101e+09,43091.20
597312,1.646103e+09,43092.36
597313,1.646103e+09,43098.94


In [12]:
data

,Unix,Date,Open,High,Low,Close,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14
0,1483233000,2017-01-01 01:10:00,965.08,966.09,962.54,962.54,-0.973221,-3.236545,-0.013736,-0.014878,-0.272070,-0.340299,-4.629180e-16
1,1483233300,2017-01-01 01:15:00,962.54,965.75,962.54,963.76,0.463215,-3.236545,-0.015608,-0.016782,-0.335879,-0.340299,-4.629180e-16
2,1483238160,2017-01-01 02:36:00,963.76,964.71,960.53,960.53,-1.238285,-1.701110,-0.020984,-0.027438,-0.285385,-0.340299,-4.629180e-16
3,1483242960,2017-01-01 03:56:00,960.53,963.64,960.53,963.00,0.943582,-2.298314,-0.021986,-0.024561,-0.335879,-0.340299,-4.629180e-16
4,1483248120,2017-01-01 05:22:00,963.00,964.04,962.80,964.04,0.394137,-1.047948,-0.021321,-0.017911,-0.335879,-0.329670,-4.629180e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
597310,1646100960,2022-03-01 02:16:00,43009.25,43247.75,42998.08,43107.80,0.840389,-1.002748,-1.102086,-1.967107,-0.335879,3.128363,-5.860503e-01
597311,1646103000,2022-03-01 02:50:00,43107.80,43137.71,43091.20,43091.20,-0.145547,-1.847266,-1.161173,-1.727761,-0.335879,-0.340299,-4.478023e-01
597312,1646103120,2022-03-01 02:52:00,43091.20,43143.52,43092.36,43143.52,0.443576,-1.286722,-1.129557,-1.299762,-0.335879,-0.185918,-2.674643e-01
597313,1646103180,2022-03-01 02:53:00,43143.52,43187.13,43098.94,43154.82,0.092791,-1.451285,-1.078015,-0.905400,-0.335879,0.818490,-1.270715e-01


In [13]:
data[0:20]

,Unix,Date,Open,High,Low,Close,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14
0,1483233000,2017-01-01 01:10:00,965.08,966.09,962.54,962.54,-0.973221,-3.236545,-0.013736,-0.014878,-0.272070,-0.340299,-4.629180e-16
1,1483233300,2017-01-01 01:15:00,962.54,965.75,962.54,963.76,0.463215,-3.236545,-0.015608,-0.016782,-0.335879,-0.340299,-4.629180e-16
2,1483238160,2017-01-01 02:36:00,963.76,964.71,960.53,960.53,-1.238285,-1.701110,-0.020984,-0.027438,-0.285385,-0.340299,-4.629180e-16
3,1483242960,2017-01-01 03:56:00,960.53,963.64,960.53,963.00,0.943582,-2.298314,-0.021986,-0.024561,-0.335879,-0.340299,-4.629180e-16
4,1483248120,2017-01-01 05:22:00,963.00,964.04,962.80,964.04,0.394137,-1.047948,-0.021321,-0.017911,-0.335879,-0.329670,-4.629180e-16
5,1483252620,2017-01-01 06:37:00,964.04,964.99,964.04,964.98,0.355496,-0.679123,-0.019482,-0.009528,-0.335879,-0.315056,-4.629180e-16
6,1483262040,2017-01-01 09:14:00,964.98,965.98,964.99,965.98,0.378051,-0.397795,-0.016668,-0.000282,-0.335879,-0.313993,-4.629180e-16
7,1483265220,2017-01-01 10:07:00,965.98,968.96,965.85,968.96,1.132725,-0.140917,-0.010673,0.015412,-0.335879,-0.261115,-4.629180e-16
8,1483265820,2017-01-01 10:17:00,968.96,970.00,968.27,970.00,0.391690,0.442296,-0.004654,0.028028,-0.335879,-0.312664,-4.629180e-16
9,1483267140,2017-01-01 10:39:00,970.00,973.14,970.00,973.14,1.188778,0.599445,0.003913,0.044767,-0.335879,-0.256864,-4.629180e-16


In [14]:
#predict_data = standard_labeling(data)
predict_data = meta_labeling(data)

TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

In [ ]:
predict_data

,Unix,Date,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,RSI-1,MACD-1,MACD_H-1,Variation-2,RSI-2,MACD-2,MACD_H-2,Variation-3,RSI-3,MACD-3,MACD_H-3,Variation-4,RSI-4,MACD-4,MACD_H-4,Target
0,1483248120,2017-01-01 05:22:00,0.394137,-1.047948,-0.021321,-0.017911,-0.335879,-0.329670,-4.629180e-16,0.943582,-2.298314,-0.021986,-0.024561,-1.238285,-1.701110,-0.020984,-0.027438,0.463215,-3.236545,-0.015608,-0.016782,-0.973221,-3.236545,-0.013736,-0.014878,1
1,1483252620,2017-01-01 06:37:00,0.355496,-0.679123,-0.019482,-0.009528,-0.335879,-0.315056,-4.629180e-16,0.394137,-1.047948,-0.021321,-0.017911,0.943582,-2.298314,-0.021986,-0.024561,-1.238285,-1.701110,-0.020984,-0.027438,0.463215,-3.236545,-0.015608,-0.016782,1
2,1483262040,2017-01-01 09:14:00,0.378051,-0.397795,-0.016668,-0.000282,-0.335879,-0.313993,-4.629180e-16,0.355496,-0.679123,-0.019482,-0.009528,0.394137,-1.047948,-0.021321,-0.017911,0.943582,-2.298314,-0.021986,-0.024561,-1.238285,-1.701110,-0.020984,-0.027438,1
3,1483265220,2017-01-01 10:07:00,1.132725,-0.140917,-0.010673,0.015412,-0.335879,-0.261115,-4.629180e-16,0.378051,-0.397795,-0.016668,-0.000282,0.355496,-0.679123,-0.019482,-0.009528,0.394137,-1.047948,-0.021321,-0.017911,0.943582,-2.298314,-0.021986,-0.024561,1
4,1483265820,2017-01-01 10:17:00,0.391690,0.442296,-0.004654,0.028028,-0.335879,-0.312664,-4.629180e-16,1.132725,-0.140917,-0.010673,0.015412,0.378051,-0.397795,-0.016668,-0.000282,0.355496,-0.679123,-0.019482,-0.009528,0.394137,-1.047948,-0.021321,-0.017911,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597304,1646100660,2022-03-01 02:11:00,-0.227913,-1.076766,-0.783202,-1.981137,-0.335879,-0.340299,-8.550598e-01,-0.169691,-1.424864,-0.617793,-1.937261,-0.578284,-1.479240,-0.439284,-1.839721,-0.045954,-1.553706,-0.235895,-1.636694,-0.336877,-1.617339,-0.079512,-1.536130,0
597305,1646100900,2022-03-01 02:15:00,-0.921795,-1.409948,-1.036978,-2.246665,3.477305,-0.340299,-6.812380e-01,-0.227913,-1.076766,-0.783202,-1.981137,-0.169691,-1.424864,-0.617793,-1.937261,-0.578284,-1.479240,-0.439284,-1.839721,-0.045954,-1.553706,-0.235895,-1.636694,1
597306,1646100960,2022-03-01 02:16:00,0.840389,-1.002748,-1.102086,-1.967107,-0.335879,3.128363,-5.860503e-01,-0.921795,-1.409948,-1.036978,-2.246665,-0.227913,-1.076766,-0.783202,-1.981137,-0.169691,-1.424864,-0.617793,-1.937261,-0.578284,-1.479240,-0.439284,-1.839721,1
597307,1646103000,2022-03-01 02:50:00,-0.145547,-1.847266,-1.161173,-1.727761,-0.335879,-0.340299,-4.478023e-01,0.840389,-1.002748,-1.102086,-1.967107,-0.921795,-1.409948,-1.036978,-2.246665,-0.227913,-1.076766,-0.783202,-1.981137,-0.169691,-1.424864,-0.617793,-1.937261,1


In [7]:
predict_data = predict_data[[i % int(max_days) == 0 for i in range(len(predict_data))]]

In [8]:
predict_data

,Unix,Date,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,RSI-1,MACD-1,MACD_H-1,Variation-2,RSI-2,MACD-2,MACD_H-2,Variation-3,RSI-3,MACD-3,MACD_H-3,Variation-4,RSI-4,MACD-4,MACD_H-4,Target
0,1483248120,2017-01-01 05:22:00,0.394137,-1.047948,-0.021321,-0.017911,-0.335879,-0.329670,-4.629180e-16,0.943582,-2.298314,-0.021986,-0.024561,-1.238285,-1.701110,-0.020984,-0.027438,0.463215,-3.236545,-0.015608,-0.016782,-0.973221,-3.236545,-0.013736,-0.014878,1
5,1483267140,2017-01-01 10:39:00,1.188778,0.599445,0.003913,0.044767,-0.335879,-0.256864,-4.629180e-16,0.391690,0.442296,-0.004654,0.028028,1.132725,-0.140917,-0.010673,0.015412,0.378051,-0.397795,-0.016668,-0.000282,0.355496,-0.679123,-0.019482,-0.009528,1
10,1483271280,2017-01-01 11:48:00,0.075673,1.984016,0.029955,0.054086,-0.335879,-0.340299,-4.629180e-16,-0.105706,1.362753,0.027616,0.059978,-0.011249,1.150590,0.024448,0.064644,0.000085,1.153541,0.019512,0.064708,0.696611,0.979150,0.012783,0.058948,1
15,1483278000,2017-01-01 13:40:00,0.537423,2.091598,0.030182,0.015142,-0.335879,-0.340299,-4.629180e-16,-0.642085,2.056611,0.030753,0.020784,-0.184922,2.786568,0.033182,0.033893,0.249447,3.025117,0.033018,0.041826,0.026535,1.911457,0.031457,0.047191,1
20,1483284660,2017-01-01 15:31:00,-0.011244,0.193415,0.028417,-0.000146,-0.335879,-0.340299,9.420333e-01,-0.700830,0.687981,0.030073,0.005210,0.641817,1.886900,0.031606,0.011503,0.052940,1.816717,0.030067,0.009360,0.287250,2.074422,0.030256,0.012311,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597285,1646096100,2022-03-01 00:55:00,0.153763,0.662725,1.634383,-0.007835,-0.335879,0.153934,-8.264876e-01,0.286728,0.746208,1.637013,-0.001227,-0.686794,0.479369,1.645833,0.027210,0.691542,0.716805,1.687199,0.168845,-0.005303,0.544185,1.587717,-0.113217,1
597290,1646097180,2022-03-01 01:13:00,-1.139000,-0.778893,1.027573,-1.432918,3.080638,-0.340299,-1.351748e+00,-0.344605,-0.185478,1.353389,-0.729133,-0.836166,0.128848,1.537976,-0.309746,-0.285112,-0.003242,1.687533,0.100305,0.574582,0.803747,1.698204,0.160159,1
597295,1646099280,2022-03-01 01:48:00,0.280423,-0.285001,0.457197,-1.257470,-0.335879,-0.340299,-1.445326e+00,-0.510515,-0.427783,0.486154,-1.477453,0.999106,0.291455,0.568796,-1.577446,0.289541,-0.287023,0.571458,-1.963134,-0.056869,-0.075476,0.752770,-1.862923,0
597300,1646099880,2022-03-01 01:58:00,-0.336877,-1.617339,-0.079512,-1.536130,0.584822,-0.340299,-1.423443e+00,-0.723558,-1.116324,0.113297,-1.291691,-0.393331,-0.750656,0.295794,-1.019758,-0.130106,-0.395380,0.390444,-0.966185,0.021761,-0.640029,0.432925,-1.069264,0


In [9]:
#corr = predict_data.corr("pearson")
#corr[["Target"]].to_clipboard()
#corr[["Target"]]

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
split_mode = "RAND"

if split_mode == "RAND":
    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.2, random_state=100)
elif split_mode == "STATIC":
    train_data = predict_data[0:int(0.66*len(predict_data))].drop(["Date", "Unix", "Target"],axis=1)
    train_labels = predict_data[0:int(0.66*len(predict_data))]["Target"]
    test_data = predict_data[int(0.66*len(predict_data)):len(predict_data)].drop(["Date", "Unix", "Target"],axis=1)
    test_labels = predict_data[int(0.66*len(predict_data)):len(predict_data)]["Target"]

In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [12]:
algo = "MLP"

if algo == "MLP":
    model = MLPClassifier(hidden_layer_sizes=(100,200,100),verbose=True)
elif algo == "RF":
    model = RandomForestClassifier(n_estimators=100,verbose=False)
elif algo == "CAT":
    model = CatBoostClassifier(iterations=100,depth=12)
elif algo == "TREE":
    model = DecisionTreeClassifier(splitter="random")
model.fit(train_data,train_labels)

Iteration 1, loss = 0.65398021
Iteration 2, loss = 0.64336664
Iteration 3, loss = 0.63996147
Iteration 4, loss = 0.63833888
Iteration 5, loss = 0.63697008
Iteration 6, loss = 0.63595846
Iteration 7, loss = 0.63490123
Iteration 8, loss = 0.63386533
Iteration 9, loss = 0.63308808
Iteration 10, loss = 0.63230100
Iteration 11, loss = 0.63149277
Iteration 12, loss = 0.63073962
Iteration 13, loss = 0.62990224
Iteration 14, loss = 0.62882980
Iteration 15, loss = 0.62831250
Iteration 16, loss = 0.62773880
Iteration 17, loss = 0.62650033
Iteration 18, loss = 0.62577357
Iteration 19, loss = 0.62499218
Iteration 20, loss = 0.62419966
Iteration 21, loss = 0.62347343
Iteration 22, loss = 0.62242316
Iteration 23, loss = 0.62162843
Iteration 24, loss = 0.62097936
Iteration 25, loss = 0.61968942
Iteration 26, loss = 0.61909913
Iteration 27, loss = 0.61817302
Iteration 28, loss = 0.61718370
Iteration 29, loss = 0.61618977
Iteration 30, loss = 0.61551841
Iteration 31, loss = 0.61454742
Iteration 32, los

c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(hidden_layer_sizes=(100, 200, 100), verbose=True)

In [50]:
model.score(test_data,test_labels)

0.5018050541516246

In [51]:
model.score(train_data,train_labels)

1.0

In [35]:
from sklearn.metrics import confusion_matrix

In [53]:
preds = model.predict(test_data)

In [54]:
conf = confusion_matrix(test_labels,preds)
conf

array([[53, 72],
       [66, 86]], dtype=int64)

In [55]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.44537815126050423

In [56]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.5443037974683544

## Cross Validation

In [19]:
def cross_validation_loop(predict_data, model_name):
    scores = []
    TPs = []
    TNs = []
    for randomize in range(1,20):
        train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=randomize)
        if model_name == "MLP":
            model = MLPClassifier(hidden_layer_sizes=(200,2000,200))
        elif model_name == "CAT":
            model = CatBoostClassifier(iterations=100,depth=12,verbose=False)
        elif model_name == "RF":
            model = RandomForestClassifier(n_estimators=100,verbose=False)
        model.fit(train_data, train_labels)
        scores.append(model.score(test_data, test_labels))
        conf = confusion_matrix(test_labels,model.predict(test_data))
        TN = conf[0,0] / (conf[0,0] + conf[1,0])
        TP = conf[1,1] / (conf[1,1] + conf[0,1])
        TPs.append(TP)
        TNs.append(TN)
        print("Score : {} || TP : {} || TN : {}".format(scores[-1],TP,TN))


    return scores,TPs,TNs

In [20]:
#scores,TPs,TNs = cross_validation_loop(predict_data,"CAT")

In [24]:
from statistics import mean

In [50]:
print("CatBoost score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

score moyen : 0.5067452023560707 || TP moyen : 0.5411620348700146 || TN moyen : 0.4619500842660272


In [22]:
scores,TPs,TNs = cross_validation_loop(predict_data,"RF")

Score : 0.51985559566787 || TP : 0.5694444444444444 || TN : 0.46616541353383456
Score : 0.49458483754512633 || TP : 0.5194805194805194 || TN : 0.4634146341463415
Score : 0.5054151624548736 || TP : 0.5465116279069767 || TN : 0.4380952380952381
Score : 0.48375451263537905 || TP : 0.5136986301369864 || TN : 0.45038167938931295
Score : 0.4693140794223827 || TP : 0.4722222222222222 || TN : 0.4639175257731959
Score : 0.48736462093862815 || TP : 0.525 || TN : 0.4358974358974359
Score : 0.51985559566787 || TP : 0.5280898876404494 || TN : 0.5050505050505051
Score : 0.49458483754512633 || TP : 0.5611510791366906 || TN : 0.427536231884058
Score : 0.49458483754512633 || TP : 0.5384615384615384 || TN : 0.4380165289256198
Score : 0.4657039711191336 || TP : 0.4899328859060403 || TN : 0.4375
Score : 0.4693140794223827 || TP : 0.5205479452054794 || TN : 0.4122137404580153
Score : 0.5306859205776173 || TP : 0.5714285714285714 || TN : 0.46788990825688076
Score : 0.4584837545126354 || TP : 0.5 || TN : 0.4

In [25]:
print("Random Forest score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

Random Forest score moyen : 0.5040851225536767 || TP moyen : 0.538149033546309 || TN moyen : 0.4600308310035195


In [26]:
scores,TPs,TNs = cross_validation_loop(predict_data,"MLP")

Score : 0.51985559566787 || TP : 0.5675675675675675 || TN : 0.46511627906976744
Score : 0.48014440433212996 || TP : 0.5068493150684932 || TN : 0.45038167938931295
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5234657039711191 || TP : 0.5496688741721855 || TN : 0.49206349206349204
Score : 0.4657039711191336 || TP : 0.46308724832214765 || TN : 0.46875
Score : 0.5342960288808665 || TP : 0.5734265734265734 || TN : 0.4925373134328358
Score : 0.5270758122743683 || TP : 0.541095890410959 || TN : 0.5114503816793893
Score : 0.5054151624548736 || TP : 0.5657894736842105 || TN : 0.432
Score : 0.48014440433212996 || TP : 0.5266666666666666 || TN : 0.4251968503937008
Score : 0.5090252707581228 || TP : 0.5302013422818792 || TN : 0.484375
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5451263537906137 || TP : 0.5933333333333334 || TN : 0.4881889763779528
Score : 0.4729241877256318 || TP : 0.5131578947368421 || TN : 0.424
Score : 0.534296028

In [27]:
print("MLP score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

MLP score moyen : 0.5078852365570967 || TP moyen : 0.5443692036536304 || TN moyen : 0.4653102613033886


# Predicting 2021 with 2019 and 2020

In [12]:
data_19_20 = predict_data.loc[(predict_data["Unix"] >= 1546300800) & (predict_data["Unix"] < 1609459200)]

In [13]:
data_19_20

,Unix,Date,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,RSI-1,MACD-1,MACD_H-1,Variation-2,RSI-2,MACD-2,MACD_H-2,Variation-3,RSI-3,MACD-3,MACD_H-3,Variation-4,RSI-4,MACD-4,MACD_H-4,Target
215320,1546300860,2019-01-01 00:01:00,-0.986145,0.491885,-0.033394,0.162775,-0.335879,-0.302567,-1.064115,0.655779,0.595083,-0.035140,0.197773,0.603821,0.319416,-0.052532,0.190523,-0.993864,-0.022621,-0.063650,0.201908,-0.227144,0.358142,-0.067437,0.240037,1
215325,1546304880,2019-01-01 01:08:00,-0.123431,0.867928,0.007335,0.114247,-0.268553,-0.340299,-0.367471,-0.086493,0.954067,0.005239,0.135973,-0.535639,0.730066,0.000480,0.154449,0.907046,1.080635,-0.007024,0.168595,0.536351,0.131097,-0.025049,0.151985,1
215330,1546308420,2019-01-01 02:07:00,-0.157800,-0.475602,-0.013818,-0.018414,-0.335879,-0.277324,-0.254652,-0.002692,-0.499192,-0.003222,0.011517,-1.705986,-0.423347,0.007805,0.050334,0.508004,0.697192,0.021669,0.108105,0.248837,0.351377,0.011937,0.103403,1
215335,1546313460,2019-01-01 03:31:00,-0.282642,-0.431510,-0.030585,-0.025066,-0.335879,-0.307350,-0.344415,0.195378,0.006301,-0.029138,-0.026620,0.121394,-0.539691,-0.031387,-0.040612,0.605738,-0.277822,-0.030769,-0.048756,-0.458200,-0.278695,-0.027670,-0.050849,0
215340,1546315500,2019-01-01 04:05:00,0.268427,-0.581463,-0.029466,0.002574,-0.335879,-0.155626,-0.545856,0.108391,-0.596378,-0.034741,-0.013984,0.209590,-0.746468,-0.036784,-0.024143,-0.079778,-0.963670,-0.037198,-0.031535,-0.203808,-1.162035,-0.033933,-0.028780,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415075,1609456920,2020-12-31 23:22:00,-0.411171,-1.060449,-0.024872,-0.300665,0.374053,-0.340299,-0.122677,-0.422850,-0.595903,0.043160,-0.154078,-0.122925,-0.285521,0.078413,-0.077695,0.278863,-0.248775,0.069796,-0.125210,0.149480,0.057241,0.042772,-0.244604,1
415080,1609457220,2020-12-31 23:27:00,-0.294028,-1.169175,-0.251386,-0.517383,-0.335879,-0.340299,0.976992,-0.096009,-1.046969,-0.189864,-0.446198,-0.276761,-1.059979,-0.144854,-0.411041,0.047760,-0.577303,-0.096913,-0.357537,0.093663,-0.853920,-0.068472,-0.354221,0
415085,1609457820,2020-12-31 23:37:00,-0.415695,-0.905861,-0.335789,-0.303776,0.418769,-0.340299,0.971806,0.306616,-0.529976,-0.314115,-0.309076,0.338154,-0.822598,-0.333325,-0.448968,-0.822252,-0.891014,-0.314479,-0.499785,0.543543,0.305953,-0.243447,-0.393199,1
415090,1609458240,2020-12-31 23:44:00,-1.184498,-1.262820,-0.471225,-0.494167,2.217967,-0.340299,1.205233,-0.386310,-0.625959,-0.331363,-0.161822,-0.192050,-0.640853,-0.294995,-0.083738,-0.105897,-0.771415,-0.292032,-0.095019,0.421466,-0.780530,-0.307576,-0.169448,1


In [14]:
data_2021 = predict_data.loc[predict_data["Unix"] >= 1609459200]

In [15]:
data_2021

,Unix,Date,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,RSI-1,MACD-1,MACD_H-1,Variation-2,RSI-2,MACD-2,MACD_H-2,Variation-3,RSI-3,MACD-3,MACD_H-3,Variation-4,RSI-4,MACD-4,MACD_H-4,Target
415100,1609459320,2021-01-01 00:02:00,-1.801269,-0.624598,-0.226616,0.194002,0.458461,-0.340299,0.118929,1.055063,0.387025,-0.133299,0.546671,-0.570242,-0.403594,-0.236590,0.346649,-0.095368,0.104788,-0.233410,0.443672,0.198281,0.346117,-0.296643,0.348470,0
415105,1609460100,2021-01-01 00:15:00,0.809332,-0.043599,-0.709988,-0.689614,-0.335879,1.465776,-1.030358,0.299717,-0.411878,-0.714952,-0.878204,-0.989780,-0.925864,-0.610454,-0.757141,-0.472230,-0.751202,-0.430818,-0.360892,-0.212690,-0.660808,-0.317558,-0.081941,0
415110,1609460820,2021-01-01 00:27:00,-0.123299,-0.692974,-0.600137,0.080846,0.165043,-0.340299,-1.347477,-0.158644,-0.571582,-0.647162,-0.052229,1.019247,-0.373510,-0.712717,-0.278974,0.309123,-0.847317,-0.805362,-0.650707,-0.903011,-0.676821,-0.785425,-0.748401,1
415115,1609461300,2021-01-01 00:35:00,0.603223,0.372150,-0.190431,0.759286,-0.335879,0.377667,-1.458820,-0.292547,0.017740,-0.296268,0.604118,0.551202,-0.568168,-0.351795,0.574150,-0.242787,-0.357744,-0.453558,0.385979,0.494891,-0.460085,-0.508106,0.304666,0
415120,1609461660,2021-01-01 00:41:00,-0.135559,1.339979,0.041359,0.612626,-0.335879,-0.065017,-0.532362,0.325123,0.692735,0.015566,0.681704,-0.163708,0.893369,-0.034093,0.690258,-0.225471,1.115213,-0.056235,0.790644,0.014218,0.627402,-0.103682,0.833644,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597285,1646096100,2022-03-01 00:55:00,0.153763,0.662725,1.634383,-0.007835,-0.335879,0.153934,-0.826488,0.286728,0.746208,1.637013,-0.001227,-0.686794,0.479369,1.645833,0.027210,0.691542,0.716805,1.687199,0.168845,-0.005303,0.544185,1.587717,-0.113217,1
597290,1646097180,2022-03-01 01:13:00,-1.139000,-0.778893,1.027573,-1.432918,3.080638,-0.340299,-1.351748,-0.344605,-0.185478,1.353389,-0.729133,-0.836166,0.128848,1.537976,-0.309746,-0.285112,-0.003242,1.687533,0.100305,0.574582,0.803747,1.698204,0.160159,1
597295,1646099280,2022-03-01 01:48:00,0.280423,-0.285001,0.457197,-1.257470,-0.335879,-0.340299,-1.445326,-0.510515,-0.427783,0.486154,-1.477453,0.999106,0.291455,0.568796,-1.577446,0.289541,-0.287023,0.571458,-1.963134,-0.056869,-0.075476,0.752770,-1.862923,0
597300,1646099880,2022-03-01 01:58:00,-0.336877,-1.617339,-0.079512,-1.536130,0.584822,-0.340299,-1.423443,-0.723558,-1.116324,0.113297,-1.291691,-0.393331,-0.750656,0.295794,-1.019758,-0.130106,-0.395380,0.390444,-0.966185,0.021761,-0.640029,0.432925,-1.069264,0


In [16]:
split_mode = "STATIC"

#if split_mode == "RAND":
#    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.2, random_state=100)
if split_mode == "STATIC":
    train_data = data_19_20.drop(["Date", "Unix", "Target"],axis=1)
    train_labels = data_19_20["Target"]
    test_data = data_2021.drop(["Date", "Unix", "Target"],axis=1)
    test_labels = data_2021["Target"]

In [17]:
%load_ext tensorboard

In [18]:
shape = (len(test_data.columns),)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=shape),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [19]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', 'Precision', 'Recall', 'AUC'])

In [20]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [21]:
model.fit(train_data, train_labels, epochs=20, validation_data=(test_data,test_labels),callbacks=[tensorboard_callback], batch_size=64)

Epoch 1/20
625/625 [==============================] - 38s 53ms/step - loss: 0.6934 - accuracy: 0.5142 - precision: 0.5180 - recall: 0.6237 - auc: 0.5203 - val_loss: 0.6931 - val_accuracy: 0.5072 - val_precision: 0.5028 - val_recall: 0.1569 - val_auc: 0.5046
Epoch 2/20
625/625 [==============================] - 28s 45ms/step - loss: 0.6922 - accuracy: 0.5166 - precision: 0.5230 - recall: 0.5455 - auc: 0.5260 - val_loss: 0.6941 - val_accuracy: 0.4934 - val_precision: 0.4928 - val_recall: 0.8939 - val_auc: 0.5079
Epoch 3/20
625/625 [==============================] - 30s 48ms/step - loss: 0.6917 - accuracy: 0.5169 - precision: 0.5261 - recall: 0.4901 - auc: 0.5296 - val_loss: 0.6964 - val_accuracy: 0.5122 - val_precision: 0.5102 - val_recall: 0.2944 - val_auc: 0.5155
Epoch 4/20
625/625 [==============================] - 31s 49ms/step - loss: 0.6913 - accuracy: 0.5200 - precision: 0.5250 - recall: 0.5739 - auc: 0.5321 - val_loss: 0.6933 - val_accuracy: 0.5035 - val_precision: 0.4976 - val_r

In [ ]:
test_loss, test_acc = model.evaluate(test_data,  test_labels, verbose=2)

#print('\nTest accuracy:', test_acc)

3816/3816 - 23s - loss: 0.6122 - accuracy: 0.7459 - precision: 0.7216 - recall: 0.8129 - 23s/epoch - 6ms/step


ValueError: too many values to unpack (expected 2)

In [40]:
time = str(datetime.date.today())
#model.save('./models/1m_200_2000_200_y2019_2020_'+time)

INFO:tensorflow:Assets written to: ./models/1m_200_2000_200_y2019_20202022-09-29\assets
